# TensorFlow crash course
### **PART 2**

### Variables

In [1]:
import tensorflow as tf

# Variables are mutable unlike tf.Tensor which provides the capability of adjusting them
tensor = tf.Variable([[1, 2, 3], [4, 5, 6]])
print(tensor)

<tf.Variable 'Variable:0' shape=(2, 3) dtype=int32, numpy=
array([[1, 2, 3],
       [4, 5, 6]], dtype=int32)>


### Variable-Assignment

In [2]:
tensor[:, 2].assign([1, 1]) 
#    ======           ===
# <Given index> <Given values> 
# Assigns the given values to the given indexes
print(tensor)

# Another way of assigment is to use (scatter_nd_update)
tensor.scatter_nd_update(indices=[[0, 0], [1, 0]], updates=[0, 0])
# The given values to the <updates> key argument are assigned to indices which the <indices> key argument got 
print(tensor)

<tf.Variable 'Variable:0' shape=(2, 3) dtype=int32, numpy=
array([[1, 2, 1],
       [4, 5, 1]], dtype=int32)>
<tf.Variable 'Variable:0' shape=(2, 3) dtype=int32, numpy=
array([[0, 2, 1],
       [0, 5, 1]], dtype=int32)>


### Common-Tensor-Structures
### *String tensor*
### *Ragged tensor*
### *Sparse tensor*
### *Tensor array*
### *Set*

### String tensor
#### *This type of structure is mostly used in natural language processing(nlp)*

In [3]:
my_string = "TensorFlow"
# Encoding
# In order to encode a tensor string, you should first calculate the ascii
# number of characters using the ord() function to have the dtype of int32
encoded_string = tf.strings.unicode_encode([ord(char) for char in my_string], "UTF-8")
print(encoded_string)
# Decoding
# The returned value will be the ascii number of characters in your string
decoded_string = tf.strings.unicode_decode(encoded_string, "UTF-8")
print(decoded_string)
# Len in string tensors
string_tensor = tf.constant(["Bear", "Tiger", "Sheep", "Cat"]) #TensorFlow automatically encodes the strings
string_length = tf.strings.length(string_tensor, unit="UTF8_CHAR")
print(string_length)

tf.Tensor(b'TensorFlow', shape=(), dtype=string)
tf.Tensor([ 84 101 110 115 111 114  70 108 111 119], shape=(10,), dtype=int32)
tf.Tensor([4 5 5 3], shape=(4,), dtype=int32)


### Ragged tensor
### *A tensor containing list of arrays with various dimensions*

In [4]:
ragged_tensor1 = tf.ragged.constant([[1, 2, 3], [4, 5], [6], []])
ragged_tensor2 =  tf.ragged.constant([[], [1], [2, 3], [4, 5, 6]]) 
print(ragged_tensor1, ragged_tensor2)
# Ragged tensor concatenation along X and Y axis
new_tensor1 = tf.concat([ragged_tensor1, ragged_tensor2], axis=0)
new_tensor2 = tf.concat([ragged_tensor1, ragged_tensor2], axis=1)
print(f"Concatenation along axis 0 {new_tensor1}", f"Concatenation along axis 1 {new_tensor2}", sep="\n|"+100*"="+"|\n")
# Ragged tensor to Regular tensor conversion 
# After the conversion zero-padding occurs on your tensor to make all the sub array shapes equivalent
print(ragged_tensor1.to_tensor(), ragged_tensor2.to_tensor())

<tf.RaggedTensor [[1, 2, 3], [4, 5], [6], []]> <tf.RaggedTensor [[], [1], [2, 3], [4, 5, 6]]>
Concatenation along axis 0 <tf.RaggedTensor [[1, 2, 3], [4, 5], [6], [], [], [1], [2, 3], [4, 5, 6]]>
|====================================================================================================|
Concatenation along axis 1 <tf.RaggedTensor [[1, 2, 3], [4, 5, 1], [6, 2, 3], [4, 5, 6]]>
tf.Tensor(
[[1 2 3]
 [4 5 0]
 [6 0 0]
 [0 0 0]], shape=(4, 3), dtype=int32) tf.Tensor(
[[0 0 0]
 [1 0 0]
 [2 3 0]
 [4 5 6]], shape=(4, 3), dtype=int32)


### Sparse tensor
### *A tensor containing zeros(similar to np.zeros)*

In [5]:
# indices : The indices in your tensor to be assigned by the given values to <values> argument
# values : Values to be assigned to the given indices
# dense_shape : shape of the tensor
sparse_tensor = tf.SparseTensor(indices=[[0, 0], [1, 1]], values=[1, 1], dense_shape=[2, 2])
# In order to visualize the created tensor, we use the tf.sparse.to_dense function
print(tf.sparse.to_dense(sparse_tensor)) #The output is supposed to be a unit matrix(np.eye(2))

tf.Tensor(
[[1 0]
 [0 1]], shape=(2, 2), dtype=int32)


### Tensor array
### *A tensor containing a list of tensors*
#### *Useful for dynamic models*

In [6]:
# The following function creates a tensor with 4 slots for our potential float32 tensors to be written in
tensor_array = tf.TensorArray(dtype=tf.float32, size=4)
# Filling the slots
#                                slot        value
#                                ==== =====================
tensor_array = tensor_array.write(0, tf.constant([1., 2.]))
tensor_array = tensor_array.write(1, tf.constant([3., 4.]))
tensor_array = tensor_array.write(2, tf.constant([5., 6.]))
tensor_array = tensor_array.write(3, tf.constant([7., 8.]))
# In order to read a slot you can use array.read(slot_number) but that's gonna pop the item out
#=============================================================================================
# To visualize a TensorArray you should use the .stack() function
print(tensor_array.stack())

tf.Tensor(
[[1. 2.]
 [3. 4.]
 [5. 6.]
 [7. 8.]], shape=(4, 2), dtype=float32)


### Sets
### *The same as python sets but in a tensor form*

In [7]:
tensor1 = tf.constant([[1, 2, 3]]) 
tensor2 = tf.constant([[3, 4, 5]])
# Union of two tensor sets
union_set = tf.sets.union(tensor1, tensor2) #A sparse tensor is returned so we use .to_dense to visualize it
print(tf.sparse.to_dense(union_set))
# Difference of two tensor sets
diff_set = tf.sets.difference(tensor1, tensor2)
print(tf.sparse.to_dense(diff_set))
# Intersection of two tensor sets
intrs_set= tf.sets.intersection(tensor1, tensor2)
print(tf.sparse.to_dense(intrs_set))

# Note : If your set contains tensors with various shapes, use zero padding 

tf.Tensor([[1 2 3 4 5]], shape=(1, 5), dtype=int32)
tf.Tensor([[1 2]], shape=(1, 2), dtype=int32)
tf.Tensor([[3]], shape=(1, 1), dtype=int32)
